In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pdb
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import backend as K
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
import helpers
from models import DaveModel, DaveVGG, DaveVGG19, SimpleModel, LeNetModel
from trainer import Trainer

Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")

In [3]:
X = helpers.get_images(train)
X_test = helpers.get_images(test)

In [4]:
y = to_categorical(train.is_iceberg.values,num_classes=2)

In [5]:
Xtr, Xv, ytr, yv = train_test_split(X, y, shuffle=False, test_size=0.20)

In [6]:
trainRunner = Trainer([
    DaveModel(Xtr, ytr, Xv, yv),
    DaveVGG(Xtr, ytr, Xv, yv),
    DaveVGG19(Xtr, ytr, Xv, yv),
    LeNetModel(Xtr, ytr, Xv, yv)
])

In [7]:
trainRunner.models[0].model.load_weights("davemodel20171203-181713.h5")
trainRunner.models[1].model.load_weights("vgg20171203-182704.h5")
trainRunner.models[2].model.load_weights("vgg1920171203-183725.h5")
trainRunner.models[3].model.load_weights("lenet20171203-184024.h5")

In [18]:
results = pd.DataFrame()

for model in trainRunner.models:
    name = model.get_name()
    print(name)
    test = model.model.predict(X_test)
    results[name] = test[:, 0]

davemodel
vgg
vgg19
lenet


In [53]:
results.head()

# results.iloc[:, 0]



,davemodel,vgg,vgg19,lenet
0,0.904778,0.506920,0.729676,0.523765
1,0.686788,0.277934,0.972092,0.611775
2,0.994779,0.248336,0.985353,0.991685
3,0.001994,0.011655,0.003505,0.004780
4,0.968177,0.280054,0.898017,0.604213


In [77]:
results["very_positive"] = results.apply(lambda x: x["davemodel"] > 0.5 and x["vgg"] > 0.5 and x["vgg19"] > 0.5 and x["lenet"] > 0.5, axis = 1)
results["very_negative"] = results.apply(lambda x: x["davemodel"] <= 0.5 and x["vgg"] <= 0.5 and x["vgg19"] <= 0.5 and x["lenet"] <= 0.5, axis = 1)
results["definitive"] = results["very_positive"] | results["very_negative"]

In [102]:
# results.head()
# results[results["definitive"]]
# results[results["very_positive"] != True & results["very_negative"] != True]

def get_score(value):
    score = 0
    
    if value > 0.5:
        score += 1
    else:
        score -= 1
        
    return score

def get_label(score):
    label = -1
    
    if score > 0:
        label = 1
    elif score < 0:
        label = 0
        
    return label

results["score"] = results.apply(lambda x: get_score(x["davemodel"]) + get_score(x["vgg"]) + get_score(x["vgg19"]) + get_score(x["lenet"]), axis = 1)

results["label"] = results.apply(lambda x: get_label(x["score"]), axis = 1)


In [113]:
X_train_pseudo = X_test[results["label"] >= 0]
y_train_pseudo = results[results["label"] >= 0]["label"]
X_train_pseudo.shape

(6984, 75, 75, 3)

In [191]:
ytr
results[results["label"] >= 0]["label"]

y_train_pseudo = np.array(pd.get_dummies(results[results["label"] >= 0]["label"]))
# y_train_pseudo = np.zeros((6984, 2))
y_train_pseudo
results[results["label"] >= 0]["label"]
# results.iloc[0]
# results[results["label"] >= 0]["label"]

test = []
stuff = results[results["label"] >= 0]["label"]

for record in stuff:
    if record == 0:
        test.append([0., 1.])
    elif record == 1:
        test.append([1., 0.])

y_train_pseudo = np.array(test)

In [192]:
print(X_train_pseudo.shape)
print(y_train_pseudo.shape)
print(Xtr.shape)
print(ytr.shape)

(6984, 75, 75, 3)
(6984, 2)
(1283, 75, 75, 3)
(1283, 2)


In [194]:
trainRunner = Trainer([
    DaveModel(X_train_pseudo, y_train_pseudo, Xv, yv),
    DaveVGG(X_train_pseudo, y_train_pseudo, Xv, yv),
    DaveVGG19(X_train_pseudo, y_train_pseudo, Xv, yv),
    LeNetModel(X_train_pseudo, y_train_pseudo, Xv, yv)
])

In [195]:
trainRunner.train(epoch = 50)

Model: davemodel
Batch Size: 32
Epochs: 50
Epoch 1/50
219/218 [==============================] - 11s - loss: 0.3113 - acc: 0.8499 - val_loss: 0.6549 - val_acc: 0.6542
Epoch 2/50
219/218 [==============================] - 10s - loss: 0.2277 - acc: 0.8975 - val_loss: 0.5179 - val_acc: 0.7414
Epoch 3/50
219/218 [==============================] - 10s - loss: 0.1926 - acc: 0.9168 - val_loss: 0.3576 - val_acc: 0.8131
Epoch 4/50
219/218 [==============================] - 9s - loss: 0.1707 - acc: 0.9249 - val_loss: 0.6313 - val_acc: 0.7072
Epoch 5/50
219/218 [==============================] - 9s - loss: 0.1644 - acc: 0.9292 - val_loss: 0.3107 - val_acc: 0.8692
Epoch 6/50
219/218 [==============================] - 9s - loss: 0.1519 - acc: 0.9349 - val_loss: 0.2900 - val_acc: 0.9003
Epoch 7/50
219/218 [==============================] - 9s - loss: 0.1464 - acc: 0.9368 - val_loss: 0.2867 - val_acc: 0.8847
Epoch 8/50
219/218 [==============================] - 9s - loss: 0.1456 - acc: 0.9398 - val_l

219/218 [==============================] - 31s - loss: 0.1538 - acc: 0.9382 - val_loss: 0.3215 - val_acc: 0.8941
Epoch 17/50
219/218 [==============================] - 31s - loss: 0.1422 - acc: 0.9431 - val_loss: 0.3080 - val_acc: 0.8660
Epoch 18/50
219/218 [==============================] - 31s - loss: 0.1290 - acc: 0.9472 - val_loss: 0.2846 - val_acc: 0.9065
Epoch 19/50
219/218 [==============================] - 31s - loss: 0.1256 - acc: 0.9502 - val_loss: 0.3326 - val_acc: 0.8910
Epoch 20/50
219/218 [==============================] - 31s - loss: 0.1275 - acc: 0.9501 - val_loss: 0.3528 - val_acc: 0.8785
Epoch 21/50
219/218 [==============================] - 31s - loss: 0.1233 - acc: 0.9528 - val_loss: 0.3825 - val_acc: 0.8723
Epoch 22/50
219/218 [==============================] - 31s - loss: 0.1298 - acc: 0.9498 - val_loss: 0.4606 - val_acc: 0.8692
Epoch 23/50
219/218 [==============================] - 31s - loss: 0.1210 - acc: 0.9539 - val_loss: 0.3470 - val_acc: 0.8972
Epoch 24/50


KeyboardInterrupt: 